In [85]:
import numpy as np
import pandas as pd
import json
import re
import os
from google.colab import drive

In [86]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [87]:
from huggingface_hub import login
from google.colab import userdata

In [88]:
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
login(token=HUGGINGFACE_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [89]:
import transformers
import torch

In [90]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    max_length=1024
)

ompound='Sc'
output_dir='/content/drive/MyDrive/Fine-tuning-llms/test_output/'

messages = [
    {"role": "system", "content": "You are a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations."},
    {"role": "user", "content": "Please, generate an input file for Sc compound."},
]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations.'}, {'role': 'user', 'content': 'Please, generate an input file for Sc compound.'}, {'role': 'assistant', 'content': "To generate a Quantum Espresso input file for a single point energy calculation of a Sc compound, let's consider Scandium Oxide (ScO) as an example.\n\nHere's a sample input file:\n\n```\n&control\n  calculation='scf'\n  restart_mode='from_scratch'\n  prefix='sc_o'\n  pseudo_dir='./pseudo'\n  outdir='./outdir'\n  verbosity='high'\n  tstress=.false.\n  ttime=.false.\n/\n\n&system\n  ibrav=4\n  celldm(1)=6.0\n  nat=2\n  ntyp=2\n  ecutwfc=30.0\n  ecutrho=240.0\n  occupations='smearing'\n  degauss=0.01\n  smearing='methfessel-paxton'\n  input_dynamics='none'\n/\n\n&electrons\n  conv_thr=1.0d-8\n  electron_maxstep=100\n  mixing_beta=0.7\n  mixing_mode='plain'\n  diagonalization='david'\n  tol_paw=1.0d-10\n/\n

In [91]:
outputs = pipeline(
    messages,
    max_new_tokens=1024,
)
print(outputs[0])
mes=outputs[0]["generated_text"][-1]
print(mes['content'])

To generate a Quantum Espresso input file for a single point energy calculation of a Sc compound, let's consider Scandium Oxide (ScO) as an example.

Here's a sample input file:

```
&control
  calculation='scf'
  restart_mode='from_scratch'
  prefix='sc_o'
  pseudo_dir='./pseudo'
  outdir='./outdir'
  verbosity='high'
  tstress=.false.
  ttime=.false.
/

&system
  ibrav=4
  celldm(1)=6.0
  nat=2
  ntyp=2
  ecutwfc=30.0
  ecutrho=240.0
  occupations='smearing'
  degauss=0.01
  smearing='methfessel-paxton'
  input_dynamics='none'
/

&electrons
  conv_thr=1.0d-8
  electron_maxstep=100
  mixing_beta=0.7
  mixing_mode='plain'
  diagonalization='david'
  tol_paw=1.0d-10
/

&ions
  ion_dynamics='bfgs'
  ion_positions
  Sc  0.0 0.0 0.0
  O   0.0 0.0 0.5
  /
  ion_velocities
  Sc  0.0 0.0 0.0
  O   0.0 0.0 0.0
  /
  ion_move=.true.
  ion_nstep=1
/

&cell
  a=6.0
  b=6.0
  c=6.0
  alpha=90.0
  beta=90.0
  gamma=90.0
  celldm(1)=6.0
  celldm(2)=6.0
  celldm(3)=6.0
  celldm(4)=6.0
  celldm(5)=6.0

In [92]:
with open('/content/drive/MyDrive/Fine-tuning-llms/data_unaries/1.in','r') as f:
    input_file1=f.read()
with open('/content/drive/MyDrive/Fine-tuning-llms/data_oxides/10.in','r') as f:
    input_file2=f.read()

In [93]:
print(input_file2)

&CONTROL
  calculation = 'scf'
  etot_conv_thr =   1.5000000000d-05
  forc_conv_thr =   5.0000000000d-05
  max_seconds =   6.8400000000d+03
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  restart_mode = 'from_scratch'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   4.5000000000d-03
  ecutrho =   6.0000000000d+02
  ecutwfc =   7.5000000000d+01
  ibrav = 0
  nat = 3
  nosym = .false.
  ntyp = 2
  occupations = 'smearing'
  smearing = 'fd'
/
&ELECTRONS
  conv_thr =   3.0000000000d-10
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
ATOMIC_SPECIES
Al     26.981538 Al.pbe-n-kjpaw_psl.1.0.0.UPF
O      15.9994 O.pbe-n-kjpaw_psl.0.1.UPF
ATOMIC_POSITIONS angstrom
Al           4.2145368473       1.4048456158       4.2145368473 
Al           4.2145368473       1.4048456158       1.4048456158 
O            0.0000000000       0.0000000000       0.0000000000 
K_POINTS automatic
33 33 33 0 0 0
CELL_PARAMETERS angstrom
      0.00000000

In [94]:
messages = [
    {"role": "system", "content": "You are a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations."},
    {"role": "user", "content": "Please, generate an input file for Sc compound. \
    The example of the input file is: " +input_file2+ "Analysise the content and generate the similar file for Sc."},
]

In [95]:
outputs = pipeline(
    messages,
    max_new_tokens=2048,
)
mes=outputs[0]["generated_text"][-1]
print(mes['content'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Analyzing the provided Quantum Espresso input file, it appears to be a configuration for a single point energy calculation of an AlO compound. Here's a breakdown of the key components:

1.  **CONTROL**: This section sets various parameters for the simulation, such as the calculation type (SCF), convergence thresholds, and output settings.
2.  **SYSTEM**: This section describes the system being simulated, including the lattice parameters, atomic species, and occupation scheme.
3.  **ELECTRONS**: This section specifies the electronic settings, such as the convergence threshold, mixing scheme, and smearing parameters.
4.  **ATOMIC_SPECIES**: This section lists the atomic species present in the simulation, along with their corresponding pseudopotentials.
5.  **ATOMIC_POSITIONS**: This section defines the initial positions of the atoms in the simulation cell.
6.  **CELL_PARAMETERS**: This section specifies the lattice vectors of the simulation cell.

To generate a similar input file for a S

Let's test "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [125]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "TinyLlama/TinyLlama_v1.1_math_code"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda",
    max_length=4096
)

compound='Sc'
output_dir='/content/drive/MyDrive/Fine-tuning-llms/test_output/'

In [98]:
with open('/content/drive/MyDrive/Fine-tuning-llms/data_unaries/1.in','r') as f:
    input_file1=f.read()
with open('/content/drive/MyDrive/Fine-tuning-llms/data_oxides/10.in','r') as f:
    input_file2=f.read()

In [115]:
messages = [
    {"role": "system", "content": "You are a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations."},
    {"role": "user", "content": "Please, generate an input file for Sc compound. \
    The example of the input file is: " +input_file2+ "Analysise the content and generate the similar file for Sc."},
]

In [139]:
text="The example of the input file is: " +input_file2+ \
     "This file describes sfc simulation of Al2O3. \
      Analysise the content and generate the similar file \
      for compound with chemical formula Cu."

In [137]:
outputs = pipeline(
    text,
    max_new_tokens=4096,
)

Both `max_new_tokens` (=4096) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [140]:
mes=outputs[0]["generated_text"]
print(mes)

The example of the input file is: &CONTROL
  calculation = 'scf'
  etot_conv_thr =   1.5000000000d-05
  forc_conv_thr =   5.0000000000d-05
  max_seconds =   6.8400000000d+03
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  restart_mode = 'from_scratch'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   4.5000000000d-03
  ecutrho =   6.0000000000d+02
  ecutwfc =   7.5000000000d+01
  ibrav = 0
  nat = 3
  nosym = .false.
  ntyp = 2
  occupations = 'smearing'
  smearing = 'fd'
/
&ELECTRONS
  conv_thr =   3.0000000000d-10
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
ATOMIC_SPECIES
Al     26.981538 Al.pbe-n-kjpaw_psl.1.0.0.UPF
O      15.9994 O.pbe-n-kjpaw_psl.0.1.UPF
ATOMIC_POSITIONS angstrom
Al           4.2145368473       1.4048456158       4.2145368473 
Al           4.2145368473       1.4048456158       1.4048456158 
O            0.0000000000       0.0000000000       0.0000000000 
K_POINTS automatic
33 33 33 0 0 0
CELL_PA

Testing "google/gemma-2-2b"

In [101]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "google/gemma-2-2b-it"


pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

compound='Sc'
output_dir='/content/drive/MyDrive/Fine-tuning-llms/test_output/'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [109]:
with open('/content/drive/MyDrive/Fine-tuning-llms/data_unaries/1.in','r') as f:
    input_file1=f.read()
with open('/content/drive/MyDrive/Fine-tuning-llms/data_oxides/10.in','r') as f:
    input_file2=f.read()

# tokenizer.chat_template = "{role}: {content}\n"
messages = [
    {"role": "user", "content": "Hi!"},
    {"role": "assistant", "content": "I am a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations."},
    {"role": "user", "content": "The example of the input file is: " +input_file2+ \
     "Analysise the content and generate the similar file for compound with formula Cu."},
]

In [110]:
outputs = pipeline(messages, max_new_tokens=4096)
response = outputs[0]["generated_text"]
print(response)


[{'role': 'user', 'content': 'Hi!'}, {'role': 'assistant', 'content': 'I am a helpful assistant specialising in generate Quantum Espresso input file for single point energy calculations.'}, {'role': 'user', 'content': "The example of the input file is: &CONTROL\n  calculation = 'scf'\n  etot_conv_thr =   1.5000000000d-05\n  forc_conv_thr =   5.0000000000d-05\n  max_seconds =   6.8400000000d+03\n  outdir = './out/'\n  prefix = 'aiida'\n  pseudo_dir = './pseudo/'\n  restart_mode = 'from_scratch'\n  tprnfor = .true.\n  tstress = .true.\n  verbosity = 'high'\n/\n&SYSTEM\n  degauss =   4.5000000000d-03\n  ecutrho =   6.0000000000d+02\n  ecutwfc =   7.5000000000d+01\n  ibrav = 0\n  nat = 3\n  nosym = .false.\n  ntyp = 2\n  occupations = 'smearing'\n  smearing = 'fd'\n/\n&ELECTRONS\n  conv_thr =   3.0000000000d-10\n  electron_maxstep = 80\n  mixing_beta =   4.0000000000d-01\n/\nATOMIC_SPECIES\nAl     26.981538 Al.pbe-n-kjpaw_psl.1.0.0.UPF\nO      15.9994 O.pbe-n-kjpaw_psl.0.1.UPF\nATOMIC_POSI

In [111]:
mes=outputs[0]["generated_text"][-1]
print(mes['content'])

```
&CONTROL
  calculation ='scf'
  etot_conv_thr =   1.5000000000d-05
  forc_conv_thr =   5.0000000000d-05
  max_seconds =   6.8400000000d+03
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  restart_mode = 'from_scratch'
  tprnfor =.true.
  tstress =.true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   4.5000000000d-03
  ecutrho =   6.0000000000d+02
  ecutwfc =   7.5000000000d+01
  ibrav = 0
  nat = 1
  nosym =.false.
  ntyp = 2
  occupations ='smearing'
  smearing = 'fd'
/
&ELECTRONS
  conv_thr =   3.0000000000d-10
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
ATOMIC_SPECIES
Cu     63.5460 Cu.pbe-n-kjpaw_psl.1.0.0.UPF
ATOMIC_POSITIONS angstrom
Cu           0.0000000000       0.0000000000       0.0000000000 
K_POINTS automatic
33 33 33 0 0 0
CELL_PARAMETERS angstrom
      0.0000000000       2.8096912316       2.8096912316
      2.8096912316       0.0000000000       2.8096912316
      2.8096912316       2.8096912316       0.0000000000
Analysise the conte